# 새로운 제스처 데이터 수집

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# 인식할 동작 목록 정의
actions = ['good', 'bad', 'four', 'five']

# 각 시퀀스의 길이와 각 동작의 지속 시간 정의
seq_length = 30
secs_for_action = 30

# MediaPipe hands 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 데이터셋을 저장할 디렉토리 생성
created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 각 동작에 대한 데이터를 수집하기 위한 루프
while cap.isOpened():
    for idx, action in enumerate(actions):
        data = [] # 현재 동작에 대한 데이터를 저장할 리스트
        
        # 웹캠에서 프레임을 읽어옴
        ret, img = cap.read()

        img = cv2.flip(img, 1) # 이미지를 좌우로 뒤집어 거울 효과

        # 수집할 동작을 표시하는 메시지 출력
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', 
                    org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000) # 데이터 수집 시작 전에 3초 대기

        start_time = time.time() # 시작 시간 기록
        
        # 지정된 기간 동안 데이터 수집
        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()

            img = cv2.flip(img, 1) # 이미지를 좌우로 뒤집음
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # 이미지를 RGB로 변환
            result = hands.process(img) # 이미지를 처리하여 손 검출
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # 이미지를 BRG로 다시 변환
        
            # 이미지에서 손이 검출된 경우
            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 4)) # 관절 좌표를 저장할 배열 초기화
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # 관절 간의 각도 계산
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3] # 관절 간의 벡터
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis] # 벡터 정규화

                    # 내적의 아크 코사인을 사용하여 각도 계산
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]
                                        
                    angle = np.degrees(angle) # 라디안을 각도로 변환
                    
                    # 각도와 동작 인덱스를 데이터에 추가
                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)
                    
                    # 이미지에 손 랜드마크 그리기
                    mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
            # 손 랜드마크가 그려진 이미지 표시
            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'): # 'q' 키를 누르면 종료
                break

        # 원시 데이터 저장
        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

        # 원시 데이터에서 시퀀스 데이터 생성
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])

        full_seq_data = np.array(full_seq_data)
        print(action, full_seq_data.shape)
        np.save(os.path.join('dataset', f'seq_{action}_{created_time}'), full_seq_data)
    break # 모든 동작에 대한 데이터 수집 후 while 루프 종료
    
# 웹캠 해제 및 모든 OpenCV 창 닫기
cap.release()
cv2.destroyAllWindows()

# 기존 K


good (826, 100)
good (796, 30, 100)
bad (885, 100)
bad (855, 30, 100)
four (881, 100)
four (851, 30, 100)
five (889, 100)
five (859, 30, 100)


In [2]:
actions = ['good', 'bad', 'four', 'five']
# 위의 기존 코드와 동일하게 데이터를 수집합니다.
# 수집한 데이터를 'dataset' 디렉토리에 저장합니다.


# 기존 데이터와 새로운 데이터 병합

In [4]:
import numpy as np

# 생성 시간 변수 설정 (예: 임의의 시간)
created_time = 1716892952

# 기존 데이터 로드
existing_actions = ['one', 'two', 'three']
existing_data = []
for action in existing_actions:
    seq_data = np.load(f'dataset/seq_{action}_{created_time}.npy')
    existing_data.append(seq_data)

existing_data = np.concatenate(existing_data, axis=0)

# 생성 시간 변수 설정 (예: 임의의 시간)
created_time = 1718526787

# 새로운 데이터 로드
new_actions = ['good', 'bad', 'four', 'five']
new_data = []
for action in new_actions:
    seq_data = np.load(f'dataset/seq_{action}_{created_time}.npy')
    new_data.append(seq_data)

new_data = np.concatenate(new_data, axis=0)

# 기존 데이터와 새로운 데이터 병합
total_data = np.concatenate([existing_data, new_data], axis=0)


# 모델 확장 및 재학습

In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

# 기존 모델 로드
model = load_model('models/model.keras')

# 입력 데이터와 레이블 준비
X = total_data[:, :, :-1]
y = total_data[:, 0, -1]

# 원-핫 인코딩
y = to_categorical(y, num_classes=len(existing_actions) + len(new_actions))

# 모델 재학습
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=32)

# 모델 저장
model.save('models/model.keras')


Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 7), output.shape=(None, 3)